In [6]:
!pip install gymnasium --quiet
!pip install 'stable-baselines3[extra]' --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 3.5 MB/s eta 0:00:00


In [10]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Reinforcement Learning

## Gymnasium

A library (https://gymnasium.farama.org/) for developing and comparing reinforcement learning algorithms, offering a wide variety of simulated environments — from classic control tasks to video games.

In [4]:
# Loading an environment with Gym
env = gym.make('FrozenLake-v1', is_slippery=False)

# Starting an Episode
state, info = env.reset()

# Sampling a Random Action
action = env.action_space.sample()

# Taking a Step in the Environment
next_state, reward, terminated, truncated, info = env.step(action)

# Closing the Environment
env.close()

### Recap of Main Interactions in Gymnasium

In [5]:
env = gym.make('FrozenLake-v1') # Create the environment
env.reset() # Start a new episode
action = env.action_space.sample() # Sample a random action
env.step(action) # Take a step in the environment
env.close() # Close the environment

## Stable Baseline3 (sb3)

A powerful library (https://stable-baselines3.readthedocs.io/en/master/) that provides reliable implementations of state-of-the-art deep reinforcement learning algorithms like DQN, PPO, A2C, and more.

In [9]:
model = DQN("MlpPolicy", env)

# Training the algorithm
model.learn(total_timesteps=10_000)

# Evaluating the algorithm
evaluate_policy(model, env)

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


(np.float64(0.5), np.float64(0.5))

In [ ]:
# Saving & Loading the algorithm
# model.save(model_path)
# model = DQN.load(model_path, env=env)

## Complete Example: Frozen lake

In [11]:
# Load environment
env = gym.make('FrozenLake-v1', map_name="4x4", is_slippery=False, render_mode=None)

# Wrap the environment to be compatible with SB3 training
env = DummyVecEnv([lambda: env])
# This changes the behaviour of the env:
# - uses and outputs vectors
# - the `.step()` combines `terminated` and `truncated` into just `done`

model_path = 'dqn_frozen_lake.pkl' # File name to save model

timesteps = 100_000 # Set training timesteps

# Train model
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log="./dqn_frozen_lake_tensorboard/")
model.learn(total_timesteps=timesteps)
model.save(model_path)
print("Model trained and saved to", model_path)

A saída de streaming foi truncada nas últimas 5000 linhas.
|    time_elapsed     | 136      |
|    total_timesteps  | 90584    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 5.08e-07 |
|    n_updates        | 22620    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9500     |
|    fps              | 663      |
|    time_elapsed     | 136      |
|    total_timesteps  | 90610    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.71e-07 |
|    n_updates        | 22627    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 9504     |
|    fps              | 663      |
|    time_elapsed     | 136    

In [12]:
# Load saved model
model = DQN.load(model_path, env=env)
print("Loaded model from", model_path)

# Evaluate the policy
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1_000, return_episode_rewards=False)
print(f"Evaluation: Mean reward = {mean_reward} ± {std_reward}")

env.close()

Loaded model from dqn_frozen_lake.pkl


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Evaluation: Mean reward = 1.0 ± 0.0


In [13]:
model = DQN.load(model_path, env=env)
print("Loaded model from", model_path)

# Recreate the env with:
# - render_mode human so we can follow along
# - is_slippery False so our player does as intended (easier to follow)
env = gym.make("FrozenLake-v1", render_mode="human", is_slippery=False)

# Reset the environment
state, info = env.reset()
state

# The model decides the next step (predict)
action, info = model.predict(state)
int(action) # 0 = left, 1 = down, 2 = right, 3 = up

# Take the step in the environment
state, reward, terminated, truncated, info = env.step(int(action))
state, reward, terminated, truncated

Loaded model from dqn_frozen_lake.pkl


/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

(1, 0, False, False)